In [1]:
import requests
from urllib.parse import urlparse
import pandas as pd
import json

In [5]:
df = pd.read_csv('./0623_2.csv')

In [6]:
# 카카오맵 API를 이용하여 주소를 위도 경도로 표현

def getLatLng(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+str(addr)
    headers = {"Authorization": "KakaoAK 8d430164aae810648e75898031ed42aa"}
    result = json.loads(str(requests.get(url,headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y']),float(match_first['x'])

In [7]:
getLatLng('서울 서초구 사임당로 58 제일빌딩')

(37.4886519231458, 127.016295331893)

In [8]:
er = [] # 에러난 행 모으기

for row in df.index:
    try:
        df.loc[row,'위도'],df.loc[row,'경도'] = getLatLng(df.loc[row,'address'])
    except:
        er.append(row)

In [9]:
df.iloc[er,:] #에러행 확인: ~~호가 붙은경우 에러발생

,Unnamed: 0,name,category,star,star_count,review,address,위도,경도
0,0,루엘드파리,"제과,베이커리",3.9,72건,267,서울 서초구 서초중앙로 18 112호,NaN,NaN
47,47,소울버킷,햄버거,4.6,12건,49,서울 서초구 서초중앙로 75 202호,NaN,NaN
92,92,동대문엽기떡볶이 교대점,떡볶이,2.3,10건,15,서울 서초구 서초중앙로 75 201호,NaN,NaN
157,157,리어카포차 서초교대1호점,실내포장마차,0.0,0건,7,서울 서초구 사임당로 39 105호,NaN,NaN
202,202,진심에취하다,일본식주점,5.0,1건,2,서울 서초구 효령로52길 35 101호,NaN,NaN
206,206,요거프레소 서울교대점,디저트카페,3.5,2건,2,서울 서초구 사임당로 38 103호,NaN,NaN
288,288,루엘드파리 아뜰리에,"제과,베이커리",0.0,0건,7,"서울 서초구 서초중앙로 18 106, 107호",NaN,NaN
313,313,명인만두 남부터미널점,분식,1.7,4건,0,서울 서초구 효령로55길 28 105호,NaN,NaN
378,378,카페다임,커피전문점,0.0,0건,1,서울 서초구 서리풀3길 24 103호,NaN,NaN
391,391,참호도과자 남부터미널점,간식,5.0,1건,0,서울 서초구 효령로 292 122호,NaN,NaN


In [10]:
er2 = []
for row in er:
    try:
        df.loc[row,'위도'],df.loc[row,'경도'] = getLatLng(df.loc[row,'address'][:-4]) #~호 제거
    except:
        er2.append(row)

In [11]:
df.iloc[er2,:] # 남은 2개  ~~호가 길어서 위에서 적용 못시킴 and 유효하지 않은 주소로 생각됨(도로명으로 바꿔준다)

,Unnamed: 0,name,category,star,star_count,review,address,위도,경도
288,288,루엘드파리 아뜰리에,"제과,베이커리",0.0,0건,7,"서울 서초구 서초중앙로 18 106, 107호",NaN,NaN
499,499,맛나스넥,분식,0.0,0건,0,서울 서초구 서초동 927-1,NaN,NaN


In [12]:
# 몇개 안되니까 직접 변경(아래는 카카오 맵에서 직접 찾아 변경)

df.loc[499,'address'] = '서울 서초구 사임당로 58 제일빌딩'
df.loc[288,'address'] = '서울 서초구 서초중앙로 18'

er3 = []
for row in er2:
    try:
        df.loc[row,'위도'],df.loc[row,'경도'] = getLatLng(df.loc[row,'address']) #~호 제거
    except:
        er3.append(row)

print(er3)

[]


In [13]:
df.info() # NaN 값이 없음을 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  500 non-null    int64  
 1   name        500 non-null    object 
 2   category    500 non-null    object 
 3   star        500 non-null    float64
 4   star_count  500 non-null    object 
 5   review      500 non-null    int64  
 6   address     500 non-null    object 
 7   위도          500 non-null    float64
 8   경도          500 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 35.3+ KB


In [14]:
df.head()

,Unnamed: 0,name,category,star,star_count,review,address,위도,경도
0,0,루엘드파리,"제과,베이커리",3.9,72건,267,서울 서초구 서초중앙로 18 112호,37.484048,127.017349
1,1,공미학,커피전문점,3.5,12건,147,서울 서초구 서초대로48길 101 1층,37.488622,127.012470
2,2,봉산옥 서초동본점,국수,3.7,79건,146,서울 서초구 반포대로8길 5-6,37.483382,127.012453
3,3,장꼬방,"찌개,전골",3.2,299건,124,서울 서초구 효령로 364,37.486893,127.023663
4,4,해담채 남부터미널점,"해물,생선",3.1,10건,47,서울 서초구 서초중앙로5길 10-8 쁘띠오피스텔 지하1층,37.485865,127.015100
